参考文档：
https://langchain-ai.github.io/langgraph/agents/streaming/

流¶
流式处理是构建响应式应用程序的关键。您需要流式传输几种类型的数据：

Agent progress — 在执行代理图中的每个节点后获取更新。
LLM tokens — 流式处理由语言模型生成的令牌。
自定义更新 — 在执行期间从工具发出自定义数据（例如，“Fetched 10/100 records”）
您可以一次流式传输多种类型的数据。

代理进度¶
要流式传输代理进度，请将 stream（） 或 astream（） 方法与 stream_mode=“updates” 一起使用。这会在每个代理步骤后发出一个事件。

例如，如果您有一个代理调用了一次工具，您应该会看到以下更新：

LLM 节点：带有工具调用请求的 AI 消息
Tool 节点：带有执行结果的工具消息
LLM 节点：最终 AI 响应

In [ ]:
# 创建llm
import os
from langchain.chat_models import init_chat_model

API_KEY = "sk-123"

BASE_URL = "https://api.deepseek.com"

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL


llm = init_chat_model("openai:deepseek-chat")

In [ ]:
# 同步调用
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def get_time():
    """
    获取当前时间
    """
    import datetime
    curr_tiem= datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return curr_tiem


tools=[get_time]

agent = create_react_agent(model=llm,tools=tools)



# 同步流式处理
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "当前时间是多少呢"}]},
    stream_mode="updates"
):
    print(chunk)
    print("\n")

    
    

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_ae5e502c-c3d6-459c-beb1-ac8a6426027b', 'function': {'arguments': '{}', 'name': 'get_time'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 74, 'total_tokens': 89, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 74}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '1a6c6fe4-b2be-46bf-9952-7edde4dfe571', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--acf6544e-04f6-4e43-b32c-dd444b47048a-0', tool_calls=[{'name': 'get_time', 'args': {}, 'id': 'call_0_ae5e502c-c3d6-459c-beb1-ac8a6426027b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 74, 'output_tokens': 15, 'total_tokens': 89, 'input_token_details': {'cache_read': 0}, 'ou

In [ ]:
#同步传输 llm的令牌 要在 LLM 生成令牌时流式传输令牌

agent = create_react_agent(
    model=llm,
    tools=[get_time],
)

for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "当前的时间是多少？"}]},
    stream_mode="messages"
):
    print("Token", token)
    print("Metadata", metadata)
    print("\n")

Token content='' additional_kwargs={} response_metadata={} id='run--83823969-fd1c-4031-b99f-afcb4d2f5ec2'
Metadata {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:e2167a9c-dfe6-f8dd-a332-13fee5a8212d', 'checkpoint_ns': 'agent:e2167a9c-dfe6-f8dd-a332-13fee5a8212d', 'ls_provider': 'openai', 'ls_model_name': 'deepseek-chat', 'ls_model_type': 'chat', 'ls_temperature': None}


Token content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0_9f6c647f-2936-44e5-b956-1e58d20bc0e7', 'function': {'arguments': '', 'name': 'get_time'}, 'type': 'function'}]} response_metadata={} id='run--83823969-fd1c-4031-b99f-afcb4d2f5ec2' tool_calls=[{'name': 'get_time', 'args': {}, 'id': 'call_0_9f6c647f-2936-44e5-b956-1e58d20bc0e7', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'get_time', 'args': '', 'id': 'call_0_9f6c647f-2936-44e5-b956-1e58d20bc0e7', 'index': 0, 'type'

In [8]:

#异步传输 llm的令牌 要在 LLM 生成令牌时流式传输令牌

agent = create_react_agent(
    model=llm,
    tools=[get_time],
)
async for token, metadata in agent.astream(
    {"messages": [{"role": "user", "content": "当前时间是多少"}]},
    stream_mode="messages"
):
    print("Token", token)
    print("Metadata", metadata)
    print("\n")

Token content='' additional_kwargs={} response_metadata={} id='run--89b903a0-885e-4504-8492-fd7f6f601c5e'
Metadata {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:d0816e8f-0bf4-6e18-ee78-e697f76c6ccc', 'checkpoint_ns': 'agent:d0816e8f-0bf4-6e18-ee78-e697f76c6ccc', 'ls_provider': 'openai', 'ls_model_name': 'deepseek-chat', 'ls_model_type': 'chat', 'ls_temperature': None}


Token content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0_dfe7ab70-9836-410b-aa07-5c33f3e2bb0a', 'function': {'arguments': '', 'name': 'get_time'}, 'type': 'function'}]} response_metadata={} id='run--89b903a0-885e-4504-8492-fd7f6f601c5e' tool_calls=[{'name': 'get_time', 'args': {}, 'id': 'call_0_dfe7ab70-9836-410b-aa07-5c33f3e2bb0a', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'get_time', 'args': '', 'id': 'call_0_dfe7ab70-9836-410b-aa07-5c33f3e2bb0a', 'index': 0, 'type'